## Data Preperation

In [138]:
import pandas as pd
import math

# Load the dataset
file_path = 'SummerStudentAdmissions2.csv'
admissions_df = pd.read_csv(file_path)

# Display the first few rows and get a summary of the dataset
admissions_df.info(), admissions_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Decision        87 non-null     object 
 1   State           88 non-null     object 
 2   GPA             87 non-null     float64
 3   WorkExp         88 non-null     float64
 4   TestScore       87 non-null     float64
 5   WritingScore    88 non-null     int64  
 6   Gender          86 non-null     float64
 7   VolunteerLevel  88 non-null     int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 5.6+ KB


(None,
   Decision       State   GPA  WorkExp  TestScore  WritingScore  Gender  \
 0    Admit  California  3.90      6.7      962.0           100     1.0   
 1    Admit     Florida  3.80      1.4      969.0            97     1.0   
 2   Banana  California  3.80      2.3      970.0            98     0.0   
 3    Admit    Colorado  3.60      0.9      969.0            97     0.0   
 4    Admit    Colorado  3.92      1.2      969.0            95    -1.0   
 
    VolunteerLevel  
 0               0  
 1               4  
 2               5  
 3               2  
 4               3  )

In [139]:
# Step 1: Handling missing values

# 1. Fill missing 'Decision' with the value from the next row
admissions_df['Decision'] = admissions_df['Decision'].fillna(method='bfill')

# 2. Fill missing 'GPA' and 'TestScore' with the median of the same 'Decision' category
for column in ['GPA', 'TestScore']:
    admissions_df[column] = admissions_df.groupby('Decision')[column].transform(
        lambda x: x.fillna(x.median())
    )

# 3. Drop rows with missing values in 'Gender'
admissions_df.dropna(subset=['Gender'], inplace=True)

# Verify missing values have been handled
missing_values = admissions_df.isnull().sum()
missing_values

/var/folders/yn/q77cltd96m39dvp38lfzz1240000gn/T/ipykernel_96893/3197201203.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  admissions_df['Decision'] = admissions_df['Decision'].fillna(method='bfill')


Decision          0
State             0
GPA               0
WorkExp           0
TestScore         0
WritingScore      0
Gender            0
VolunteerLevel    0
dtype: int64

In [140]:
# Step 2 (Finalized): Reapplying your custom instructions for clarity and consistency

# Check unique values in 'Decision' to identify invalid entries
unique_decisions = admissions_df['Decision'].unique()

# Replace the invalid entry 'Banana' with the value from the next row
admissions_df['Decision'] = admissions_df['Decision'].mask(admissions_df['Decision'] == 'Banana').fillna(method='bfill')

# Replace invalid gender entries (-1) with 1
admissions_df['Gender'] = admissions_df['Gender'].replace(-1, 1)

# Verify corrections
corrected_decisions = admissions_df['Decision'].unique()
corrected_genders = admissions_df['Gender'].unique()

corrected_decisions, corrected_genders


/var/folders/yn/q77cltd96m39dvp38lfzz1240000gn/T/ipykernel_96893/902899856.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  admissions_df['Decision'] = admissions_df['Decision'].mask(admissions_df['Decision'] == 'Banana').fillna(method='bfill')


(array(['Admit', 'Decline', 'Waitlist'], dtype=object), array([1., 0.]))

In [141]:
# Step 3: Convert Data Types

# Convert 'Decision' and 'Gender' to categorical types with meaningful labels
admissions_df['Decision'] = admissions_df['Decision'].astype('category')
admissions_df['Gender'] = admissions_df['Gender'].map({0: 'Female', 1: 'Male'}).astype('category')

# Verify the data types and category conversion
admissions_df.info(), admissions_df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, 0 to 87
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Decision        86 non-null     category
 1   State           86 non-null     object  
 2   GPA             86 non-null     float64 
 3   WorkExp         86 non-null     float64 
 4   TestScore       86 non-null     float64 
 5   WritingScore    86 non-null     int64   
 6   Gender          86 non-null     category
 7   VolunteerLevel  86 non-null     int64   
dtypes: category(2), float64(3), int64(2), object(1)
memory usage: 5.1+ KB


(None,
   Decision       State   GPA  WorkExp  TestScore  WritingScore  Gender  \
 0    Admit  California  3.90      6.7      962.0           100    Male   
 1    Admit     Florida  3.80      1.4      969.0            97    Male   
 2    Admit  California  3.80      2.3      970.0            98  Female   
 3    Admit    Colorado  3.60      0.9      969.0            97  Female   
 4    Admit    Colorado  3.92      1.2      969.0            95    Male   
 
    VolunteerLevel  
 0               0  
 1               4  
 2               5  
 3               2  
 4               3  )

In [142]:
# Step 4: Outlier Detection

# Detect outliers in GPA, TestScore, and WorkExp
outliers_gpa = admissions_df[(admissions_df['GPA'] < 0.0) | (admissions_df['GPA'] > 4.0)]
outliers_testscore = admissions_df[(admissions_df['TestScore'] < 400) | (admissions_df['TestScore'] > 1600)]  # Assuming SAT scale
outliers_workexp = admissions_df[admissions_df['WorkExp'] < 0]  # Negative work experience shouldn't exist

# Count of outliers
outliers_gpa_count = outliers_gpa.shape[0]
outliers_testscore_count = outliers_testscore.shape[0]
outliers_workexp_count = outliers_workexp.shape[0]


# Detect outliers in WritingScore (assuming a valid range of 0-100)
outliers_writingscore = admissions_df[(admissions_df['WritingScore'] < 20) | (admissions_df['WritingScore'] > 100)]


def replace_writingscore_outlier(row):
    if row['WritingScore'] < 20 or row['WritingScore'] > 100:
        decision_type = row['Decision']
        avg_writingscore = admissions_df[admissions_df['Decision'] == decision_type]['WritingScore'].mean()
        return math.ceil(avg_writingscore)  # Round up the average score
    else:
        return row['WritingScore']

admissions_df['WritingScore'] = admissions_df.apply(replace_writingscore_outlier, axis=1)

# Verify no more WritingScore outliers
remaining_outliers_writingscore = admissions_df[
    (admissions_df['WritingScore'] < 20) | (admissions_df['WritingScore'] > 100)
].shape[0]

remaining_outliers_writingscore, outliers_gpa_count, outliers_testscore_count, outliers_workexp_count




(0, 1, 0, 0)

In [143]:
# Handle the GPA outlier by replacing it with the average GPA of the same decision type
def replace_gpa_outlier(row):
    if row['GPA'] > 4.0 or row['GPA'] < 0.0:
        decision_type = row['Decision']
        avg_gpa = admissions_df[admissions_df['Decision'] == decision_type]['GPA'].mean()
        return math.ceil(avg_gpa)
    else:
        return row['GPA']

admissions_df['GPA'] = admissions_df.apply(replace_gpa_outlier, axis=1)

# Verify no more GPA outliers
remaining_outliers_gpa = admissions_df[(admissions_df['GPA'] < 0.0) | (admissions_df['GPA'] > 4.0)].shape[0]
remaining_outliers_gpa


0

In [144]:
# Step 5: Feature Engineering

# Create 'TotalScore' by summing 'TestScore' and 'WritingScore'
admissions_df['TotalScore'] = admissions_df['TestScore'] + admissions_df['WritingScore']

# Create 'ExperienceLevel' based on 'WorkExp'
def experience_level(years):
    if years <= 2:
        return 'Low'
    elif years <= 5:
        return 'Medium'
    else:
        return 'High'

admissions_df['ExperienceLevel'] = admissions_df['WorkExp'].apply(experience_level).astype('category')

# Verify the new features
admissions_df[['TotalScore', 'ExperienceLevel']].head()


,TotalScore,ExperienceLevel
0,1062.0,High
1,1066.0,Low
2,1068.0,Medium
3,1066.0,Low
4,1064.0,Low


In [145]:
# Capitalize the first letter of each state for consistency
admissions_df['State'] = admissions_df['State'].str.title()

# Sort the dataset by 'Decision', then 'State', then 'GPA', and then 'WorkExp'
admissions_df = admissions_df.sort_values(by=['Decision', 'State', 'GPA', 'WorkExp']).reset_index(drop=True)

# Display a grouped view to verify sorting
#grouped_view = admissions_df.groupby(['Decision', 'State']).head(10)  # Showing top 10 from each group for verification

# Save the sorted dataset
sorted_file_path = 'Cleaned_Sorted_SummerStudentAdmissions.csv'
admissions_df.to_csv(sorted_file_path, index=False)

#grouped_view.head(), sorted_file_path